**Date: 24th Aug 2021** \
**Author: Osama Sidahmed**

### Objective

The objective of this notebook is to create a classifier that differentiate between a tweet related to a real natural disaster and others that do not, using Natural language Processing (NLP) methods.

the data will be explored after which it it will be processed and then few classifiers using standard Machine learning algorithms and Deep Learning algorithms will be trained on the data t get to the best comparison using the relevant evaluation metrics.

### EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

**Dataset Summary:**

This has been downloaded form [Kaggle](https://www.kaggle.com/vstepanenko/disaster-tweets). 
The description below is from the same page.

--------------
Description:

Context
The file contains over 11,000 tweets associated with disaster keywords like “crash”, “quarantine”, and “bush fires” as well as the location and keyword itself. The data structure was inherited from Disasters on social media

The tweets were collected on Jan 14th, 2020.

Some of the topics people were tweeting:

The eruption of Taal Volcano in Batangas, Philippines
Coronavirus
Bushfires in Australia
Iran downing of the airplane flight PS752

Disclaimer: The dataset contains text that may be considered profane, vulgar, or offensive.

Inspiration
The intention was to enrich the already available data for this topic with newly collected and manually classified tweets.
The initial source Disasters on social media which is used in Real or Not? NLP with Disaster Tweets competition on Kaggle.


The first step is to read the csv file into a data frame and explore the dataframe different fields.

In [5]:
# read dataframe
df = pd.read_csv('data/tweets.csv', index_col='id')

In [7]:
# show dataframe
df

,keyword,location,text,target
id,,,,
0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0
...,...,...,...,...
11365,wrecked,Blue State in a red sea,Media should have warned us well in advance. T...,0
11366,wrecked,arohaonces,i feel directly attacked 💀 i consider moonbin ...,0
11367,wrecked,🇵🇭,i feel directly attacked 💀 i consider moonbin ...,0


Some quick observations about the data: \
    - In addition to the id column that seems to be unique, the data consists of four (4) other columns, `keyword`, `location`, `text`, and `target`
    


### Pre-prcessing

### Modelling